In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np

from pasna_analysis import Experiment, Embryo, Trace

experiment_name = '20240515'
exp_path = Path.cwd().parent.joinpath('data',experiment_name)
exp = Experiment(exp_path)

In [ ]:
'''Removes all embryos where the first peak was detected before 30
minutes.'''

FIRST_PEAK_THRESHOLD = 30*60
activities = exp.activities()
lengths = exp.lengths()

embryos = [Embryo(a,l) for a, l in zip(activities, lengths)]
traces = {e.name: None for e in embryos}

to_remove = []
for emb in embryos:
    time, act, stct = emb.activity[:, 0], emb.activity[:, 1], emb.activity[:, 2], 
    trace = Trace(time, act, stct)
    if trace.get_first_peak_time() < FIRST_PEAK_THRESHOLD:
        print(f'First peak detected before 30 mins of imaging for {emb.name}.')
        to_remove.append(emb.name)
    else:
        traces[emb.name] = trace

embryos = [e for e in embryos if e.name not in to_remove]

In [ ]:
embryo = embryos[2]

fig, ax = plt.subplots(figsize=(8,6))

time = embryo.activity[:, 0]/60
trace = traces[embryo.name]

dff = trace.dff
time = time[:trace.trim_idx]

savgol = trace.savgol
savgol1 = trace.savgol1
savgol2 = trace.savgol2

ax.plot(time, dff, label='dF/F')
ax.plot(time, savgol, label='savgol')
ax.plot(time, savgol1, label='savgol1')
ax.plot(time, savgol2, label='savgol2')
ax.set_xlabel('Time [min]')
ax.legend()

for x in trace.peak_idxes:
    ax.scatter(time[x], savgol[x], color='k')

In [ ]:
'''Visualizing peak values, developmental time at peak value times, and 
the relationship between dff and dev time.'''
fig, (ax, ax2, ax3, ax4) = plt.subplots(4, figsize=(10,16))
for emb in embryos:
    print(emb.name)
    time = emb.activity[:, 0]
    trace = traces[emb.name]
    first_peak_time = trace.get_first_peak_time()

    time = time[:trace.trim_idx]
    dev_time = emb.get_DT_from_time(time)

    first_peak_dev = emb.get_DT_from_time(first_peak_time)

    interval = time > (first_peak_time - 300)
    ax.plot(time, trace.dff)
    ax2.plot(time[interval] - first_peak_time, trace.dff[interval])

    interval = (dev_time > first_peak_dev - 0.01)
    rng = np.logical_and(dev_time > 1.9, dev_time < 2.5)
    ax3.plot(dev_time[rng], trace.dff[rng])
    ax4.plot(dev_time[interval] - first_peak_dev, trace.dff[interval])

ax.set_title('Dff x time')
ax2.set_title('Dff x time, aligned by first peak')
ax3.set_title('Dff x developmental time')
ax4.set_title('Dff x developmental time, aligned by first peak')
fig.tight_layout()

In [ ]:
'''What was the developmental time in the beginning of the movie and 
before hatching?'''
fig, (ax, ax2) = plt.subplots(2)

peak_times = []
dev_firsts = []
firsts = []
dev_ends = []
ends = []

dev_metrics = {'dev_fp': [], 'time_fp': [], 'dev_ht': [], 'time_ht': []}
for emb in embryos:
    trace = traces[emb.name]
    time = emb.activity[:, 0]

    time = time[:trace.trim_idx]
    time_first_peak = trace.peak_times[0]

    dev_time_first_peak = emb.get_DT_from_time(time_first_peak)
    dev_time_end = emb.get_DT_from_time(time[-1])

    dev_metrics['dev_fp'].append(dev_time_first_peak)
    dev_metrics['time_fp'].append(time_first_peak)
    dev_metrics['dev_ht'].append(dev_time_end)
    dev_metrics['time_ht'].append(time[-1])

ax.scatter(np.zeros_like(dev_metrics['dev_fp']), dev_metrics['dev_fp'], label='dev_first')
print(np.std(dev_metrics['dev_fp'])/np.average(dev_metrics['dev_fp']))
ax.scatter(np.ones_like(dev_metrics['dev_ht']), dev_metrics['dev_ht'], label='dev_end')
print(np.std(dev_metrics['dev_ht'])/np.average(dev_metrics['dev_ht']))
ax2.scatter(np.zeros_like(dev_metrics['time_fp']), dev_metrics['time_fp'], label='first')
print(np.std(dev_metrics['time_fp'])/np.average(dev_metrics['time_fp']))
ax2.scatter(np.ones_like(dev_metrics['time_ht']), dev_metrics['time_ht'], label='end')
print(np.std(dev_metrics['time_ht'])/np.average(dev_metrics['time_ht']))

ax.set_title('Developmental time')
ax.legend()

In [ ]:
'''Developmental time at first episode and before hatching.'''
dev_metrics = {'dev_fp': [], 'time_fp': [], 'dev_ht': [], 'time_ht': []}

for emb in embryos:
    time = emb.activity[:, 0]

    trace = traces[emb.name]
    time = time[:trace.trim_idx]
    first_peak = trace.peak_times[0]

    dvt = emb.developmental_time()
    dev_metrics['dev_fp'].append(emb.get_DT_from_time(first_peak))
    dev_metrics['time_fp'].append(first_peak)
    dev_metrics['dev_ht'].append(emb.get_DT_from_time(time[-1]))
    dev_metrics['time_ht'].append(time[-1])

fig, ax = plt.subplots()
ax.scatter(dev_metrics['time_fp'], dev_metrics['dev_fp'],  label='t=first peak')
ax.scatter(dev_metrics['time_ht'], dev_metrics['dev_ht'],  label='t=before hatching')

for fp, tfp, ht, tht in zip(*dev_metrics.values()):
    ax.plot((tfp, tht), (fp, ht), color='k', linewidth=0.3)

ax.set_title('developmental time')
ax.legend()

fig.tight_layout()